In [69]:
import openpyxl
import pandas as pd
import numpy as np
import re # patteern matching

### 1. Gradient Boosting (XGBoost)
reference source: https://machinelearningmastery.com/xgboost-for-time-series-forecasting/ 

We need to first change the data into a more usable and compact form. This approach will be purely data driven, and will only use enrollment data from past Fall and Spring semesters to make next Fall semester's prediction.

XG Boost will be run on each core class independently rather than doing a multivariate approach for simplicity and perhaps better results?

Here we are first filtering the data for only classes we are interested in, and making a dictionary where the key is the course name and the value is another dictionary where the key is semester (Season + Year) and the value is the count of students in that class in that semester

In [60]:
# will augment 200 with 261 and 220 with 262 as they are comparable
cs_classes = ["CSCI261_Pass_C_Sem", "CSCI262_Pass_Sem", "CSCI200_Pass_C_Sem", "CSCI220_Pass_Sem", "CSCI306_Pass_Sem", "CSCI341_Pass_Sem", "CSCI358_Pass_Sem", "CSCI400_Pass_Sem", "CSCI406_Pass_Sem", "CSCI442_Pass_Sem"]
raw_data = pd.read_excel('CS Course Enrollment Data 030824.xlsx')
new_df = raw_data.loc[:, cs_classes] # filter for only classes we want
new_df.head()

# Create dictionaries of counts of students in each class for each semester listed
column_counts = {}
# Iterate over each column in the DataFrame
for column in new_df.columns:
    # Get counts of unique values in the column
    value_counts = new_df[column].value_counts()
    
    # Convert value_counts Series to dictionary
    counts_dict = value_counts.to_dict()
    
    # Store the dictionary of counts for the column
    column_counts[column] = counts_dict

#augment 200 with 261 and 220 with 262 and remove 261 and 262 from dictionary
column_counts.get("CSCI200_Pass_C_Sem").update(column_counts.get("CSCI261_Pass_C_Sem"))
column_counts.get("CSCI220_Pass_Sem").update(column_counts.get("CSCI262_Pass_Sem"))
del column_counts["CSCI261_Pass_C_Sem"]
del column_counts["CSCI262_Pass_Sem"]

# Print the dictionaries for each column
# 2008 appears to be the earliest, and it goes up to Fall 2023
for course, counts_dict in column_counts.items():
    print(f"Counts for {course}:")
    print(counts_dict)
    print()


Counts for CSCI200_Pass_C_Sem:
{'Fall 2023': 302, 'Spring 2023': 241, 'Fall 2022': 15, 'Summer 2023': 41, 'Fall 2020': 289, 'Fall 2019': 255, 'Spring 2021': 249, 'Fall 2018': 236, 'Spring 2019': 213, 'Spring 2020': 183, 'Fall 2021': 173, 'Spring 2022': 168, 'Spring 2018': 136, 'Fall 2017': 112, 'Summer 2020': 83, 'Fall 2016': 56, 'Spring 2017': 55, 'Spring 2016': 45, 'Summer 2021': 43, 'Summer 2022': 39, 'Spring 2015': 30, 'Fall 2015': 26, 'Fall 2014': 18, 'Spring 2014': 14, 'Summer 2019': 13, 'Summer One 2018': 12, 'Fall 2013': 8, 'Summer Two 2016': 7, 'Spring 2012': 7, 'Spring 2013': 6, 'Spring 2011': 5, 'Fall 2012': 5, 'Fall 2011': 4, 'Fall 2009': 4, 'Summer Two 2015': 4, 'Spring 2010': 4, 'Summer One 2017': 4, 'Summer Two 2012': 2, 'Fall 2007': 2, 'Summer Two 2014': 1, 'Summer 2009': 1, 'Fall 2008': 1}

Counts for CSCI220_Pass_Sem:
{'Fall 2023': 217, 'Spring 2023': 1, 'Fall 2022': 11, 'Summer 2023': 16, 'Fall 2021': 254, 'Fall 2019': 207, 'Fall 2020': 192, 'Spring 2019': 169, 'Spri

Next we want to sort the dictionaries in order to easily create our time-series data. We also remove the summer semesters from the data as enrollment numbers for summer are very different and likely do not vastly alter enrollment for the following Fall semester.

In [66]:
# Define a custom sorting key function
def custom_sort_key(s):
    # Extract the year from the string
    pattern = r'\b\d{4}\b'
    year = int(re.search(pattern, s).group())
    # Assign a sort order based on the season ("Spring" comes before "Fall")
    season_order = 0 if s.startswith("Spring") else 1
    # Combine the year and season order to create the sorting key
    return (year, season_order)

# sort dictionaries and remove summer semesters
for course, counts_dict in column_counts.items():
    print(f"Counts for {course}:")
    counts_dict = dict(sorted(counts_dict.items(), key = lambda item: custom_sort_key(item[0])))
    summer = r'^Summer'
    counts_dict = {key: value for key, value in counts_dict.items() if not re.match(summer, key)}
    column_counts[course] = counts_dict
    print(counts_dict)
    print()

Counts for CSCI200_Pass_C_Sem:
{'Fall 2007': 2, 'Fall 2008': 1, 'Fall 2009': 4, 'Spring 2010': 4, 'Spring 2011': 5, 'Fall 2011': 4, 'Spring 2012': 7, 'Fall 2012': 5, 'Spring 2013': 6, 'Fall 2013': 8, 'Spring 2014': 14, 'Fall 2014': 18, 'Spring 2015': 30, 'Fall 2015': 26, 'Spring 2016': 45, 'Fall 2016': 56, 'Spring 2017': 55, 'Fall 2017': 112, 'Spring 2018': 136, 'Fall 2018': 236, 'Spring 2019': 213, 'Fall 2019': 255, 'Spring 2020': 183, 'Fall 2020': 289, 'Spring 2021': 249, 'Fall 2021': 173, 'Spring 2022': 168, 'Fall 2022': 15, 'Spring 2023': 241, 'Fall 2023': 302}

Counts for CSCI220_Pass_Sem:
{'Fall 2011': 4, 'Spring 2012': 1, 'Fall 2012': 5, 'Spring 2013': 3, 'Fall 2013': 2, 'Spring 2014': 2, 'Fall 2014': 7, 'Spring 2015': 9, 'Fall 2015': 12, 'Spring 2016': 21, 'Fall 2016': 31, 'Spring 2017': 45, 'Fall 2017': 52, 'Spring 2018': 89, 'Fall 2018': 128, 'Spring 2019': 169, 'Fall 2019': 207, 'Spring 2020': 119, 'Fall 2020': 192, 'Spring 2021': 165, 'Fall 2021': 254, 'Spring 2022': 131, '

Finally, we want to create separate data arrays using these dictionaries that can be fed easily into the algorithm. We will use a function to turn the time series data into supervised data. We will just use CSCI200 as a test and if successful, we can try the others.

In [73]:
print(column_counts.get("CSCI200_Pass_C_Sem))
csci200 = list(column_counts.get("CSCI200_Pass_C_Sem").values())
csci200 = np.array(csci200)
print(csci200)

[  2   1   4   4   5   4   7   5   6   8  14  18  30  26  45  56  55 112
 136 236 213 255 183 289 249 173 168  15 241 302]
30
